### 지도 학습
- 입력과 함께 정답을 알려주고 그 정답을 맞추도록 하는 학습 방법

### 비지도 학습
- 정답의 제공 없이 학습 데이터로부터 유용한 정보를 추출하는 학습 방법

### 학습 매개변수
- 학습 과정에서 변화하는 매개변수, 이 값이 변화하면 알고리즘 출력이 변화

### 손실함수
- 알고리즘이 얼마나 잘못하고있는지를 표현하는 지표

### 무차별 대입법 Brute-Force
- 가능한 모든 수를 대입해보는 방법
- 최적값이 존재하는 범위를 알아야 함
- 계산복잡도가 매우 높음

### 경사하강법 Gradient Descent

### 최적화 이론
- 손실 함수을 가장 작게(또는 크게) 하는 입력값을 찾는다.
- 가능한 모든 해 중 최적의 해를 찾는 문제를 해결하는 이론
- 부등식 및 등식 제약 조건을 지키면서 목적 함수가 최소가 되게 하는 x를 찾는 문제
- minimize $f(x)$
- subject to $g_i(x)<=0, h_j(x)=0$
- saddle point : 기울기가 0이 되지만 극값이 아닌 지점
- Momentum : Local minimum과 잡음에 대처
- Adagrad : 학습이 많이 된 변수는 학습률을 감소
- RMSProp
- Adam : RMSProp + Momentum

# 경사 하강법을 이용한 얕은 신경망 학습


In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
- 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
            
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [4]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) # df(x)/dx
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)

## 데이터셋 생성, 전처리

In [5]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## 모델 생성

In [6]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [9]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 2.2644705772399902, Accuracy: 17.299999237060547
Epoch 2, Loss: 1.8606008291244507, Accuracy: 42.39999771118164
Epoch 3, Loss: 1.6281869411468506, Accuracy: 52.20000076293945
Epoch 4, Loss: 1.4573979377746582, Accuracy: 63.400001525878906
Epoch 5, Loss: 1.326873779296875, Accuracy: 69.5999984741211
Epoch 6, Loss: 1.2200862169265747, Accuracy: 77.70000457763672
Epoch 7, Loss: 1.1333603858947754, Accuracy: 76.30000305175781
Epoch 8, Loss: 1.0606375932693481, Accuracy: 81.5999984741211
Epoch 9, Loss: 0.9876958727836609, Accuracy: 81.0
Epoch 10, Loss: 0.9326199293136597, Accuracy: 83.0999984741211
Epoch 11, Loss: 0.8738253116607666, Accuracy: 83.0999984741211
Epoch 12, Loss: 0.8471482396125793, Accuracy: 84.69999694824219
Epoch 13, Loss: 0.8035646080970764, Accuracy: 85.79999542236328
Epoch 14, Loss: 0.7685476541519165, Accuracy: 85.0
Epoch 15, Loss: 0.7287571430206299, Accuracy: 85.0999984741211
Epoch 16, Loss: 0.6995693445205688, Accuracy: 86.29999542236328
Epoch 17, Loss:

## 데이터셋 및 학습 파라미터 저장

In [10]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                    W_h=W_h,
                    b_h=b_h,
                    W_o=W_o,
                    b_o=b_o)